In [7]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dill as pickle

from matplotlib import animation, rc
rc('animation', html='html5')

import sys
sys.path.append('..')
from util import add_angles, angle_between, angled_vector, clip_angle, unit_vector, x_axis
import binning

In [11]:
# TODO: Handling of boundary conditions is buggy!
class Fish(object):
    def __init__(self):
        self.kick_position_start = np.array([5,5])
        self.kick_position_end = np.array([5,5])
        
        self.kick_time_start = 0
        self.kick_time_end = 0
        self.time = 0
        
    def kick(self, boundary_condition):
        #print(f"Kick of at {self.time}")
        # TODO: Replace with proper kick model
        kick_duration = 5 #s
        kick_length = 4 #cm
        kick_trajectory = (angled_vector(np.deg2rad(np.random.random()*45)) *
                           kick_length)
        
        self.kick_time_start = self.time
        self.kick_time_end = self.time + kick_duration
        self.kick_position_start = boundary_condition(self.kick_position_end)
        self.kick_position_end = self.kick_position_start + kick_trajectory
    
    def step(self, dt, boundary_condition=lambda x:x):
        eps = 1e-6 # to compare floats
        while dt > eps:
            # Move until length of current kick
            cur_time = min(self.kick_time_end, self.time + dt)
            #print(dt, cur_time, self.time)
            dt -= cur_time - self.time
            self.time = cur_time
            #print(cur_time, dt)

            if dt > eps:
                # We need to kick off here.
                self.kick(boundary_condition=boundary_condition)
        
    def get_pos(self,boundary_condition=lambda x:x):
        # TODO: Use exponential interpolation here
        pos_dt = self.kick_position_end - self.kick_position_start
        kick_time = self.kick_time_end - self.kick_time_start
        elapsed_time = self.time - self.kick_time_start
        
        if kick_time == 0:
            weight = 0
        else:
            weight = elapsed_time/kick_time
        return boundary_condition((1-weight) * self.kick_position_start
                            + weight *self.kick_position_end)
    
    def get_state(self, boundary_condition=lambda x:x):
        position = self.get_pos(boundary_condition=boundary_condition)
        
        kick_trajectory = self.kick_position_end - self.kick_position_start
        # TODO: Double check headings, is wrong!
        heading_change = angle_between(x_axis, kick_trajectory)
        heading = angle_between(x_axis, kick_position_end)
        print(heading_change,heading)
        kick_length = np.linalg.norm(kick_trajectory)
        return {'heading_change': heading_change,
                'length': kick_length,
                #'dt': 0,
                'angle_f0': np.pi,
                'angle_f1': 0,
                'x_f0': 2,
                'y_f0': 2,
                'x_f1': 2,
                'y_f1': 3}

class WorldState(object):
    def __init__(self, size):
        self.size = size
        self.time = 0
        self.time_dt = 0.01 # time step from experiment
        self.history_dt = 5 # frames between history snapshots
        self.history_size = 11 # keep n snapshots
        self.fish = [Fish(), Fish()]
        self.bc = lambda pos: np.mod(pos, self.size)
    
    def step(self, dt=None):
        if dt is None:
            dt = self.time_dt
        self.time += dt
        # Move simulation forward
        for f in self.fish:
            f.step(dt, boundary_condition=self.bc)
    
    def get_pos(self):
        return np.array([self.fish[0].get_pos(self.bc), 
                        self.fish[1].get_pos(self.bc)])
    
world = WorldState(size=np.array([10,10]))

f = Fish()
#bc = lambda pos: np.mod(pos, np.array([10,10]))
#f.get_pos(), f.step(1, bc), f.get_pos(),f.step(2.99, bc), f.get_pos(), f.step(0.01, bc), f.get_pos()
f.step(5), f.kick_position_start, f.kick_position_end, 
f.get_state()

0.22729076700852183


{'angle_f0': 3.141592653589793,
 'angle_f1': 0,
 'dt': 0,
 'heading_change': array([3.89712186, 0.9013552 ]),
 'length': 1,
 'x_f0': 2,
 'x_f1': 2,
 'y_f0': 2,
 'y_f1': 3}

In [40]:
from itertools import product
with open('../adaptive_bins.model', 'rb') as f:
    edges = pickle.load(f)
# Generate data frame that receptive field expects from kick history
history = [{'heading_change': np.deg2rad(180),
           'length': 1,
           'dt': 0,
           'angle_f0': np.pi,
           'angle_f1': 0,
           'x_f0': 2,
           'y_f0': 2,
           'x_f1': 2,
           'y_f1': 3}
          ]
# Set all wall distances to infinity (no walls here)
wall_distance_columns = [f'wall_distance{i}_f{id}' for i, id in product(range(0,4), range(0,2))]
ß
for step in history:
    for col in wall_distance_columns:
        step[col] = float('inf')
df = pd.DataFrame(history*4)
binning.transform_coords_df(df, cutoff_wall_range=0)

,dt,trajectory_f0_x,trajectory_f0_y,x_f1,y_f1,trajectory_f1_x,trajectory_f1_y
0,0.0,-1.0,1.224647e-16,2.220446e-16,-1.0,-1.0,-1.224647e-16
1,0.0,-1.0,1.224647e-16,2.220446e-16,-1.0,-1.0,-1.224647e-16
2,0.0,-1.0,1.224647e-16,2.220446e-16,-1.0,-1.0,-1.224647e-16
3,0.0,-1.0,1.224647e-16,2.220446e-16,-1.0,-1.0,-1.224647e-16


In [3]:
def add_to_buffer(buffer, value):
    buffer_local = np.roll(buffer, shift=-1)
    buffer_local[-1] = value
    np.copyto(dst=buffer, src=buffer_local)
    return buffer

fig = plt.figure(figsize=(10,10))
ax = plt.axes(xlim=(0, world.size[0]), ylim=(0, world.size[1]))
plt.close(fig)

lines = [None] * 2
lines[0], = ax.plot([], [], c='red', linewidth=5, label='fish 1')
lines[1], = ax.plot([], [], c='green', linewidth=5, label='fish 2')

ax.legend(loc='upper right')

# Set up animation buffers.
visible_steps = 10

# Shape of buffer: fish_id, coord, step
animation_buffer = np.zeros((2,2,visible_steps)) + (world.size/2)[0]

animation_frames = 1000
animation_interval = 200
animation_dt = animation_interval/1000

def init():
    for line in lines:
        line.set_data([], [])
    return lines[0], lines[1]

def animate(i):
    world.step(animation_dt)
    cur_positions = world.get_pos()
    #print(cur_positions)
    for fish_id, position in enumerate(cur_positions):
        # Update animation buffers
        add_to_buffer(animation_buffer[fish_id, 0], position[0])
        add_to_buffer(animation_buffer[fish_id, 1], position[1])
        
        # Update graphic
        lines[fish_id].set_data(animation_buffer[fish_id, 0],
                                animation_buffer[fish_id, 1])

    return lines[0], lines[1], 

anim = animation.FuncAnimation(fig,
                               animate,
                               init_func=init,
                               frames=animation_frames,
                               interval=animation_interval,
                               blit=True)
anim